# Cap 3 - Loop Closure

In the context of SLAM (Simultaneous Localization and Mapping), loop closure refers to the process of recognizing a previously visited location. This recognition helps correct cumulative errors in the map and the robot's estimated position.

Imagine a robot navigating an environment: as it moves, small errors in its movement measurements can accumulate over time, leading to an inaccurate map. When the robot detects a place it has seen before (a "loop"), it can use this information to adjust its map and position estimates, effectively "closing the loop" and reducing errors.

<img src="imgs/loopclosure.gif">

The process consist in 2 steps:

- Feature detection (Scale Invariant Feature Transform - SIFT, SURF, ORB, ...);
- Feature matching (Bag of Words - BoW, RANSAC, ...).

In [3]:
import pandas as pd
import numpy as np

In [4]:
######### Prepare data
sample_i = 1
df = pd.read_csv(f"./sample_data/t_x_y_z{sample_i}.csv", names=["time","x","y","z"], header=None)
# Split the DataFrame into separate clouds
def select_frame(i):
    start_time = df.time.values[0]
    low_limit = (i-1)*0.1666+start_time
    high_limit = i*0.1666+start_time
    return df[(df['time']>low_limit) & (df['time']<high_limit)]

df2 = select_frame(1)
# n = How many blocks of cloud_size equals 1st frame size fits on the dataset?
n = int(df.index.values[-1]/len(df2))
print(f"{n} frames with {len(df2)} points each")

dfs = [select_frame(i) for i in range(1,n)]
print(f"{len(dfs)} frames with almost {len(df)//len(dfs)} points each")

# Remove x=0, y=0 points
for i in range(len(dfs)):
    idx_to_remove = dfs[i].loc[(dfs[i]['x']==0)].index.values
    dfs[i] = dfs[i].drop(index=idx_to_remove)
    idx_to_remove = dfs[i].loc[(dfs[i]['y']==0)].index.values
    dfs[i] = dfs[i].drop(index=idx_to_remove)
cloud_count = [dfs[i].x.count() for i in range(len(dfs))]
print(f"{len(dfs)} frames with almost {sum(cloud_count)//len(dfs)} points each")

# Remove time column
for i in range(len(dfs)):
    try:
        del dfs[i]['time']
    except:
        pass

# to numpy
dfs2 = []
for i in range(len(dfs)):
    dfs2.append(dfs[i].to_numpy())

161 frames with 664 points each
160 frames with almost 668 points each
160 frames with almost 618 points each


#### Functions

In [ ]:
# https://github.com/RainerKuemmerle/g2o/blob/master/g2o/examples/ba_anchored_inverse_depth/ba_anchored_inverse_depth_demo.cpp

from collections import defaultdict

import g2o
import numpy as np

pixel_noise = 1
outlier_ratio = 0
robust_kernel = False
schur_trick = False

def invert_depth(x):
    assert len(x) == 3 and x[2] != 0
    return np.array([x[0], x[1], 1]) / x[2]


def main():
    optimizer = g2o.SparseOptimizer()
    if schur_trick:
        solver = g2o.BlockSolverSE3(g2o.LinearSolverEigenSE3())
    else:
        solver = g2o.BlockSolverX(g2o.LinearSolverEigenX())  # slower
    solver = g2o.OptimizationAlgorithmLevenberg(solver)
    optimizer.set_algorithm(solver)

    true_points = np.hstack(
        [
            np.random.random((500, 1)) * 3 - 1.5,
            np.random.random((500, 1)) - 0.5,
            np.random.random((500, 1)) + 3,
        ]
    )

    focal_length = 1000.0
    principal_point = (320, 240)
    cam = g2o.CameraParameters(focal_length, principal_point, 0)
    cam.set_id(0)

    optimizer.add_parameter(cam)

    true_poses = []
    num_pose = 15
    for i in range(num_pose):
        # pose here means transform points from world coordinates to camera coordinates
        pose = g2o.SE3Quat(np.identity(3), [i * 0.04 - 1, 0, 0])
        true_poses.append(pose)

        v_se3 = g2o.VertexSE3Expmap()
        v_se3.set_id(i)
        v_se3.set_estimate(pose)
        if i < 2:
            v_se3.set_fixed(True)
        optimizer.add_vertex(v_se3)

    point_id = num_pose
    inliers = dict()
    sse = defaultdict(float)

    for i, point in enumerate(true_points):
        visible = []
        for j, pose in enumerate(true_poses):
            z = cam.cam_map(pose * point)
            if 0 <= z[0] < 640 and 0 <= z[1] < 480:
                visible.append((j, z))
        if len(visible) < 2:
            continue

        v_p = g2o.VertexPointXYZ()
        v_p.set_id(point_id)
        v_p.set_marginalized(schur_trick)

        anchor = visible[0][0]
        point2 = true_poses[anchor] * (point + np.random.randn(3))
        if point2[2] == 0:
            continue
        v_p.set_estimate(invert_depth(point2))
        optimizer.add_vertex(v_p)

        inlier = True
        for j, z in visible:
            if np.random.random() < outlier_ratio:
                inlier = False
                z = np.random.random(2) * [640, 480]
            z += np.random.randn(2) * pixel_noise

            edge = g2o.EdgeProjectPSI2UV()
            edge.resize(3)
            edge.set_vertex(0, v_p)
            edge.set_vertex(1, optimizer.vertex(j))
            edge.set_vertex(2, optimizer.vertex(anchor))
            edge.set_measurement(z)
            edge.set_information(np.identity(2))
            if robust_kernel:
                edge.set_robust_kernel(g2o.RobustKernelHuber())

            edge.set_parameter_id(0, 0)
            optimizer.add_edge(edge)

        if inlier:
            inliers[point_id] = (i, anchor)
            error = (
                true_poses[anchor].inverse() * invert_depth(v_p.estimate())
                - true_points[i]
            )
            sse[0] += np.sum(error**2)
        point_id += 1

    print("Performing full BA:")
    optimizer.initialize_optimization()
    optimizer.set_verbose(True)
    optimizer.optimize(10)

    for i in inliers:
        v_p = optimizer.vertex(i)
        v_anchor = optimizer.vertex(inliers[i][1])
        error = (
            v_anchor.estimate().inverse() * invert_depth(v_p.estimate())
            - true_points[inliers[i][0]]
        )
        sse[1] += np.sum(error**2)

    print("\nRMSE (inliers only):")
    print("before optimization:", np.sqrt(sse[0] / len(inliers)))
    print("after  optimization:", np.sqrt(sse[1] / len(inliers)))


main()

Performing full BA:

RMSE (inliers only):
before optimization: 1.7197515324056674
after  optimization: 0.5095616135652088
